In [1]:
import pandas as pd
from tkinter import *
import tkinter as ttk
import os
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
#get data from entry field, workbook name
def show_message():
    global directory
    global workbook_name 
    directory = entry.get()
    workbook_name = entry2.get()
    root.destroy()

# create a window with the input of workbook name
root = Tk()
root.title("Inputs")
root.geometry("400x150+500+300") 

lbl_1 = Label(root, text='Enter a file directory:')
lbl_1.place(x = 10, y = 10) 

entry = ttk.Entry(width=60)
entry.place(x = 10, y = 30) 
#defauld entry text
entry.insert(0, os.getcwd())

lbl_2 = Label(root, text='Enter an original file name:')
lbl_2.place(x = 10, y = 60) 

entry2 = ttk.Entry(width=60)
entry2.place(x = 10, y = 80)
#defauld entry text
entry2.insert(0, 'in_staff_raw_data.xlsx')

btn = ttk.Button(text="Start", command=show_message)
btn.place(x = 10, y = 110) 

root.mainloop()

In [3]:
print('Doing the math, wait a minute...')

Doing the math, wait a minute...


### Calculation

In [4]:
#create a python directiry format
directory = directory.replace("\\","/")+'/'
file_inputs = 'Inputs.xlsx'

#upload files
inputs_fx = pd.read_excel(directory+file_inputs, sheet_name='FX', header=0)
inputs_ranges = pd.read_excel(directory+file_inputs, sheet_name='Ranges', header=0)
staff_raw_data = pd.read_excel(directory+workbook_name, header=1)

In [5]:
print('Staff_raw_data has been uploaded')

Staff_raw_data has been uploaded


In [6]:
#create a table of FX incl. all currencies for correction with LE
list_fx_le = inputs_fx[inputs_fx['Legal Entity'].notnull()]
#create a list incl. all currencies for correction without LE
list_fx_wo_le = inputs_fx[inputs_fx['Legal Entity'].isnull()]['Currency'].to_list()

In [7]:
#create a copy of raw data to make changes
staff_corr = staff_raw_data.copy()

In [8]:
#create a dataframe containing adjustable currencies wo LE filter
staff_fx_part_1 = staff_corr[staff_corr['Currency'].isin(list_fx_wo_le)]
#create a dataframe containing adjustable currencies with LE filter
staff_fx_part_2 = staff_corr.merge(list_fx_le, on =['Currency', 'Legal Entity'], how='inner')

In [9]:
#create a dataframe containing all adjustable currencies
staff_fx = pd.concat([staff_fx_part_1, staff_fx_part_2])

In [10]:
#create a list of columns in ranges:
hours_columns = list(range(inputs_ranges.loc[inputs_ranges['Range']=='Hours']['First column'].values[0] - 1, 
                      inputs_ranges.loc[inputs_ranges['Range']=='Hours']['Last column'].values[0]))

original_curr_columns = list(range(
    inputs_ranges.loc[inputs_ranges['Range']=='Original currency values']['First column'].values[0] - 1, 
                      inputs_ranges.loc[inputs_ranges['Range']=='Original currency values']['Last column'].values[0]))

usd_curr_columns = list(range(
    inputs_ranges.loc[inputs_ranges['Range']=='USD currency values']['First column'].values[0] - 1, 
                      inputs_ranges.loc[inputs_ranges['Range']=='USD currency values']['Last column'].values[0]))

In [11]:
#create a dataframe with a reversal numbers 
staff_fx_reversal = staff_fx.copy()
staff_fx_reversal.iloc[:, hours_columns + original_curr_columns + usd_curr_columns] = staff_fx.iloc[:,
                                            hours_columns + original_curr_columns + usd_curr_columns] * -1

In [12]:
#Create a dataframe containing adjusted currencies (original in USD)
staff_fx_adj = staff_fx.copy()
staff_fx_adj.iloc[:, original_curr_columns] = staff_fx_adj.iloc[:, usd_curr_columns]
staff_fx_adj['Currency'] = 'USD'
staff_fx_adj['FX Rate'] = 1

In [13]:
#consolidate dataframes
staff_fx_combined = pd.concat([staff_corr, staff_fx_reversal, staff_fx_adj])

### Process FBU names 1-to-1 FP-FBU relationship

In [14]:
#add data from MDS
mds = pd.read_excel(directory+'in_MDS.xlsx', header=0)

In [15]:
#keep only FP and FBU GUIDs
mds_2 = mds[['BusinessUnit_Id', 'EDBU_id', 'ED_FBU_Name']].drop_duplicates().reset_index(drop=True).fillna('N/A').rename(
    columns={'EDBU_id':'FBU_GUID_mds', 'BusinessUnit_Id':'FP_GUID_mds', 'ED_FBU_Name':'FBU_Name_mds'})
#Add count to identify 1-to-1 FP to FBU or 1-to-many FP to FBU)
mds_2['Count'] = mds_2.groupby('FP_GUID_mds')['FP_GUID_mds'].transform('count')

In [16]:
print('MDS data frame has been uploaded')

MDS data frame has been uploaded


In [17]:
#add data from BU_params
bu_params = pd.read_excel(directory+'in_BU_Params.xlsx', sheet_name='BU Param', header=0)

In [18]:
print('BU_params data frame has been uploaded')

BU_params data frame has been uploaded


In [19]:
#keep only FBU GUID and FBU name
bu_params = bu_params[['BusinessUnits guid', 'BusinessUnits']].drop_duplicates().reset_index(drop=True).rename(
    columns={'BusinessUnits guid':'FBU_GUID_params', 'BusinessUnits':'FBU_name_params'})

In [20]:
#Add data from BU_params
mds_3 = mds_2.merge(bu_params, left_on='FBU_GUID_mds', right_on='FBU_GUID_params', how='left')
#Create mapping FP_GUID - FBU_name. keep only FP GUIDs mapped 1-to-1 to FBU (remove FP GUIDs mapped 1-to-many FBU)
FBU_names_1to1 = mds_3.loc[mds_3['Count']==1].drop(['Count', 'FBU_GUID_mds', 'FBU_GUID_params', 'FBU_Name_mds'], axis='columns')
FBU_names_1to1['mapping_type'] = '1-to-1 FP-FBU relationship'

In [21]:
#Identify incorrect FBU names in staff df
staff_fx_combined['Correct_FBU_name'] = staff_fx_combined['Business Unit'].isin(
    set(bu_params['FBU_name_params']))
#add correct FBU names from BU params
staff_fx_combined_2 = staff_fx_combined.merge(FBU_names_1to1, left_on='Fin Project GUID', right_on='FP_GUID_mds', how='left')

### Fuzzy lookup 1-to-many FP-FBU relationship

In [22]:
#Create mapping keeping FP GUIDs mapped 1-to-many FBU, for fuzzy lookup
FBU_names_fuzzy = mds_3.loc[mds_3['Count']!=1].drop(['Count', 'FBU_GUID_mds', 'FBU_GUID_params', 'FBU_Name_mds'], axis='columns')

In [23]:
#Create a list of FBU from staff data that contains only FBU with 1-to-many relationship for FP-FBU
fuzzy_FBU = (staff_fx_combined_2.loc[(staff_fx_combined_2['Correct_FBU_name']==False) & (staff_fx_combined_2['FBU_name_params']
             .isnull())][['Business Unit', 'Fin Project GUID']]
             .drop_duplicates()
             .reset_index(drop=True))

In [24]:
#Add FBU_name_params for Fuzzy lookup 
fuzzy_FBU_2 = (fuzzy_FBU.merge(FBU_names_fuzzy, left_on='Fin Project GUID', right_on='FP_GUID_mds', how='left')
               .drop(['Fin Project GUID'], axis='columns'))

In [33]:
#Find Levenshtein Distance to calculate the differences between FBU names
try:
    fuzzy_FBU_2['fuzz_ratio'] = (fuzzy_FBU_2.apply(
        lambda x: fuzz.ratio(x['Business Unit'], x['FBU_name_params']), axis=1))
except:
    fuzzy_FBU_2['fuzz_ratio'] = []

In [34]:
# Group by 'FP_GUID_mds' and find the index of the max 'fuzz_ratio' for each group
idx = fuzzy_FBU_2.groupby('FP_GUID_mds')['fuzz_ratio'].idxmax()
# Filter the DataFrame to keep only the rows with the max 'fuzz_ratio' in each group
FBU_names_1to_many = fuzzy_FBU_2.loc[idx].drop(['Business Unit', 'fuzz_ratio'], axis='columns')
FBU_names_1to_many['mapping_type'] = '1-to-many FP-FBU relationship'

### Combine mappings and replace FBU names

In [36]:
#combine both mappings into one data frame
FBU_names_final = pd.concat([FBU_names_1to1, FBU_names_1to_many], ignore_index=True)

In [37]:
#Remove columns in order to add it again from the combined mapping
staff_fx_combined_3 = staff_fx_combined_2.drop(['FP_GUID_mds', 'FBU_name_params', 'mapping_type'], axis='columns')
#add correct FBU names from combined mapping
staff_fx_combined_4 = staff_fx_combined_3.merge(FBU_names_final, left_on='Fin Project GUID', right_on='FP_GUID_mds', how='left')

In [43]:
#replace values of incorrect FBU
def replace_business_unit(row):
    if not row['Correct_FBU_name'] and pd.notna(row['FBU_name_params']):
        return row['FBU_name_params']
    else:
        return row['Business Unit']
staff_fx_combined_4['Business Unit'] = staff_fx_combined_4.apply(replace_business_unit, axis=1)
staff_fx_combined_4 = staff_fx_combined_4.drop(['Correct_FBU_name', 'FP_GUID_mds', 'FBU_name_params', 'mapping_type'], axis='columns')

Uncomment the row above to remove additional columns

In [39]:
print('Saving the finished staff_wo_iwo file...')

Saving the finished staff_wo_iwo file...


In [40]:
#save to output file to excel
staff_fx_out = staff_fx_combined_4.copy()
staff_fx_out.to_excel(directory + 'out_staff_wo_iwo.xlsx', index=False)

### Check

In [41]:
#Calculate sum of USD amount before and after the correction
Amount_USD_before = round(staff_raw_data['Total Salaries_USD'].sum(), 1)
Amount_USD_after = round(staff_fx_out['Total Salaries_USD'].sum(), 1)
#Calculate sum of work hours amount before and after correction
Amount_Hours_before = round(staff_raw_data['Work hours'].sum(), 1)
Amount_Hours_after = round(staff_fx_out['Work hours'].sum(), 1)
#Calculate how many new rows were created in out data frame compare to raw_data df
New_rows = staff_fx_out.shape[0] - staff_raw_data.shape[0]

In [42]:
window = Tk() 
#create a title
window.title('Result')
#set the size of the window and the place on the screen
window.geometry("300x200+500+300") 
#creating labels
label_1 = Label(window, text='Amount USD before: '+str(Amount_USD_before)).place(x = 30, y = 10) 
label_2 = Label(window, text='Amount USD after: '+str(Amount_USD_after)).place(x = 30, y = 30) 
label_3 = Label(window, text='Difference: '+str(Amount_USD_after-Amount_USD_before)).place(x = 30, y = 50) 

label_4 = Label(window, text='Amount of Work hours before: '+str(Amount_Hours_before)).place(x = 30, y = 80) 
label_5 = Label(window, text='Amount of Work hours after: '+str(Amount_Hours_after)).place(x = 30, y = 100) 
label_6 = Label(window, text='Difference: '+str(Amount_Hours_after-Amount_Hours_before)).place(x = 30, y = 120) 

label_7 = Label(window, text='New rows have been created: '+str(New_rows)).place(x = 30, y = 150) 

window.mainloop() 